In [1]:
!pip install langchain torch transformers datasets chromadb sentence-transformers accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.

In [2]:
import torch
from datasets import load_dataset
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [3]:
from huggingface_hub import notebook_login

# Logining Hugging Face
notebook_login()

In [4]:
# Import necessary libraries
from transformers import AutoTokenizer, pipeline

# Set up HuggingFace Pipeline with facebook/bart-base model
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "text-generation",  # task
    model=model_name,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,  # Adjust data type based on your hardware
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

#LLM intialized in HuggingFace Pipeline wrapper
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
data = load_dataset("manasuma/ml_interview_qa",split="train")

Generating train split:   0%|          | 0/161 [00:00<?, ? examples/s]

In [6]:
data = data.to_pandas()
data.to_csv("dataset.csv")
data.head()

,Questions,Answers
0,Difference between SuperVised and Unsupervise...,Supervised learning is when you know the outco...
1,What is Reinforcment Learning and how would y...,A learning differs from supervised learning in...
2,What is Deep Learning ?,Deep learning is defined as algorithms inspire...
3,Difference between Machine Learning and Deep ...,Since DL is a subset of ML and both being subs...
4,Difference between Bias and Variance?,Bias is definned as over simpliciation assumpt...


In [7]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='/content/dataset.csv')
data = loader.load()

In [8]:
# Split document into text chunks
text_splitter =RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [9]:
# Initialize the open-source embedding function, default: text-embedding-ada-002
embedding_function =  HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Load it into ChromaDB
db = Chroma.from_documents(data, embedding_function)

In [23]:
#Design Prompt Template
template = """You are a interviewer chatbot for a students.

{context}
" "
Question:

Answer: """

In [27]:
from langchain.prompts import PromptTemplate
# Initialize prompt using prompt template via LangChain
prompt = PromptTemplate(template=template, input_variables=["context"])

# Format and print the initialized prompt
print(
    prompt.format(
        context="A student is preparing for their final exams"
    )
)

You are a interviewer chatbot for a students.

A student is preparing for their final exams
" "
Question:

Answer: 


In [25]:
#Chain to have all components together and query the LLM
chain_type_kwargs = {"prompt": prompt}

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [26]:
import textwrap
import os
# Formatted printing
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=80)))

#Running chain through LLM with query
query = query = "I'm preparing for a machine learning engineer interview. Can you tell me the top 5 most asked questions?"
response = chain.run(query)
print_response(response)


You are a interviewer chatbot for a students.  : 98 Questions: 1) What's the
trade-off between bias and variance? [[src](http://houseofbots.com/news-
detail/2849-4-data-science-and-machine-learning-interview-questions)] Answers:
If our model is too simple and has very few parameters then it may have high
bias and low variance. On the other hand if our model has large number of
parameters then it’s going to have high variance and low bias. So we need to
find the right/good balance without overfitting and underfitting the data.
[[src]](https://towardsdatascience.com/understanding-the-bias-variance-
tradeoff-165e6 " " Question:  Answer:   2) What is the difference between
supervised and unsupervised learning? [[src](http://houseofbots.com/news-
detail/2849-4-data-science-and-machine-learning-interview-questions)] Answers:
Supervised learning involves training a model on labeled data to make
predictions on new, unseen data. Unsupervised learning involves training a model
on unlabeled data 